In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

In [ ]:
xtest = pd.read_csv("XTest.csv")

xtrain = pd.read_csv("xTrain.csv")
ytrain = pd.read_csv("yTrain.csv")

xtrain.head()

In [ ]:
#This drops any and all values where the most common element appears more than 90% of the time as this information probably isn't the most useful.

labels = list(xtrain.columns.values)
for elem in labels:
    ##If 90% (.9) or more of a column have the same value, ie, a column with the values[0,0,0,0,0,0,0,0,0,1], then the column will be dropped. This value can be changed if needed.
    if (list(xtrain[elem].value_counts(normalize = True))[0]) > 0.9:
        xtrain.drop(columns = elem, inplace=True)

#This drops our test data to 94 columns.
xtrain.head()

In [ ]:
# Create the Scaler object
scaler = MinMaxScaler()
scaled_xtrain = pd.DataFrame(scaler.fit_transform(xtrain), columns=xtrain.columns)

In [ ]:
print(scaled_xtrain)

In [ ]:
import matplotlib.pyplot as plt
import math
import random

In [ ]:
#Residual Sum of Squares
def SSE(w,data):
    totalError=0
    for i in range(data.shape[0]): 
        yOutput = m*data[i,0]+b
        y = data[i,1]
        error = (y-yOutput)**2
        totalError =totalError+ error
    return totalError

In [ ]:
def stochastic_gradient_descent_step(xtrain, ytrain):
    indexes = np.random.randint(low=0, high=xtrain.shape[0], size=1000)
    xstoch = np.take(xtrain,indexes,0)
    ystoch = np.take(ytrain,indexes,0)
    w = np.linalg.pinv(xstoch).dot(ystoch)
    l_rate = 0.025
    augX = np.concatenate((np.ones((xstoch.shape[0],1)),xstoch), axis=1)
    
    for i in range(10):
        indexes = np.random.randint(low=0, high=xtrain.shape[0], size=1000)
        xstoch = np.take(xtrain,indexes,0)
        ystoch = np.take(ytrain,indexes,0)
        grad = 2 * (augX.T.dot(augX.dot(w)-ystoch) / xstoch.shape[0])
        w = w - l_rate * grad
    
    return w

In [ ]:
def sample_data(data, batch_size):
    #mini-batch gradient descent
    if(batch_size< data.shape[0]):
        #the first two line are simulating like if we were choosing randomly points from the data
        index = np.random.permutation(data.shape[0]) #first suffle the index of data
        index = index[0:batch_size] #then we take the batch 
        #algorithm for getting the sample_data
        data_sample=[]
        for i in index:
            data_sample.append(data[i])
        return np.array(data_sample)

In [ ]:

returned = stochastic_gradient_descent_step(xtrain, ytrain)

print(returned)
"""max_epochs = 100

#RUN CODE
#start = time.time()
for i in range(max_epochs):
    data_sample = sample_data(np.array(X),50)
    w = stochastic_gradient_descent_step(w,data_sample)
    sse = SSE(w,np.array(X))
    #if(i%10==0):
        #end = time.time()
        #print('time consumtion = ',end-start)
        #print('iteration ', i)
        #start = time.time()
    #print('At step %d - Line: y = %.2fx + %.2f - Error: %.2f' %(i+1,m,b,sse))
print('\nBest  line: y = %.2fx + %.2f - Error: %.2f' %(w, sse))
"""

In [ ]:
#Plot of all variables which weren't dropped above and their correlation to each other. As seen there are 2 rectangles of correlation.
plt.subplots(figsize=(5,5))
sns.heatmap(xtrain.corr())

In [ ]:
print("hi")

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error

accuracy_list = []

xtrain = pd.read_csv("xTrain.csv")
ytrain = pd.read_csv("yTrain.csv")
xtest = pd.read_csv("xTest.csv")
#Scale xtrain
scaler = MinMaxScaler()
xtrain = pd.DataFrame(scaler.fit_transform(xtrain), columns=xtrain.columns)
ytrain = pd.DataFrame(scaler.fit_transform(ytrain), columns=ytrain.columns)
xtest = pd.DataFrame(scaler.fit_transform(xtest), columns=xtest.columns)
scaler.fit_transform(ytrain)

#This drops any and all values where the most common element appears more than 90%
#of the time as this information probably isn't the most useful.

labels = list(xtrain.columns.values)
for elem in labels:
    ##If 90% (.9) or more of a column have the same value.
    #then the column will be dropped. This value can be changed if needed.
    if (list(xtrain[elem].value_counts(normalize = True))[0]) > 0.98:
        xtrain.drop(columns = elem, inplace=True)
        xtest.drop(columns = elem, inplace=True)

# transform data
poly = PolynomialFeatures(2)
xtrain = poly.fit_transform(xtrain)
xtest = poly.fit_transform(xtest)


#randomly select 1/5th of the data
indexes = np.random.randint(low=0, high=xtrain.shape[0], size=30000)
xstoch = np.take(xtrain,indexes,0)
ystoch = np.take(ytrain,indexes,0)

#w = np.linalg.pinv(xstoch).dot(ystoch)
w = xstoch.T.dot(xstoch)

#simulate identity matrix regularization (not enough memory for full)
# (Xt*X - gamma*I) but with a 0 in the top left corner of the matrix
gamma = 0.2
for i in range(1, w.shape[0]):
    w[i,i] += gamma
   
#linear regression closed form with regularization
w = np.linalg.inv(w).dot(xtrain.T.dot(ytrain))

#learning rate
l_rate = 0.01

#create list of all indexes
indexes = np.arange(0, xtrain.shape[0])
np.random.shuffle(indexes)

steps = 20000
epochs = 2
for i in range(epochs):

    #repeat until epoch is over
    index = 0
    while ((indexes.shape[0] - index) > steps):
        #select a number(steps) random entries from the index list and access them
        xstoch = np.take(xtrain,indexes[index:index+steps],0)
        ystoch = np.take(ytrain,indexes[index:index+steps],0)
       
        #perform gradient descent calculation
        grad = 2 * (xstoch.T.dot(xstoch.dot(w)-ystoch) / xstoch.shape[0])
        w = w - l_rate * grad
       
        #increment search space
        index+=steps
       
    #when the epoch finishes, reset the index list
    indexes = np.arange(0, xtrain.shape[0])
    np.random.shuffle(indexes)
    l_rate-=0.0001
    ypred = xtrain.dot(w)
    accuracy_list.append(mean_absolute_error(ytrain,ypred))

import xlsxwriter 
row = 1
output = xtest.dot(w)
print(output)
workbook = xlsxwriter.Workbook('output.xlsx') 
worksheet = workbook.add_worksheet()  
worksheet.write(0, 0, 'Index')
worksheet.write(0, 1, 'Label') 
for i in range(len(xtest)):
    worksheet.write(row, 0, i)
    worksheet.write(row, 1, output[i])
    row += 1

workbook.close()

ValueError: non-broadcastable output operand with shape (7624,1) doesn't match the broadcast shape (7624,280)

In [ ]:
ypred = xtrain.dot(w)
print("Training error is ", sklearn.metrics.mean_absolute_error(ytrain,ypred),"%.")

In [1]:
print(accuracy_list)

NameError: name 'accuracy_list' is not defined